In [269]:
import numpy as np

In [389]:
class Expression:
    def __init__(self, ele_func, sub_expr1, sub_expr2=None):
        self._ele_func  = ele_func
        self._sub_expr1 = sub_expr1
        self._sub_expr2 = sub_expr2
    
    def evaluation_at(self, val_dict):
        
        # self._sub_expr2 is None implies that self._ele_func is an unary operator
        if self._sub_expr2 is None: 
            return self._ele_func.evaluation_at(
                self._sub_expr1, val_dict)
        
        # self._sub_expr2 not None implies that self._ele_func is a binary operator
        else:
            return self._ele_func.evaluation_at(
                self._sub_expr1, self._sub_expr2, val_dict)
    
    def derivative_at(self, var, val_dict,order=1):
        
        if var is self: return 1.0
        
        # sub_expr2 is None implies that _ele_func is an unary operator
        if self._sub_expr2 is None:
            return self._ele_func.derivative_at(
                self._sub_expr1, var, val_dict,order)
        
        # sub_expr2 not None implies that _ele_func is a binary operator
        else:
            return self._ele_func.derivative_at(
                self._sub_expr1, self._sub_expr2, var, val_dict,order)
    
    def __neg__(self):
        return Expression(Neg, self)

                
    def __add__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, self, another)
        # if the other operand is not an Expression, then it must be a number
        # the number then should be converted to a Constant
        else:
            return Expression(Add, self, Constant(another))
    
    
    def __radd__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, another, self)
        else:
            return Expression(Add, Constant(another), self)
    
    def __sub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, self, another)
        else:
            return Expression(Sub, self, Constant(another))
    
    def __rsub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, another, self)
        else:
            return Expression(Sub, Constant(another), self)
        

    def __mul__(self, another):
        if isinstance(another, Expression):
            return Expression(Mul,self,another)
        else:
            return Expression(Mul, self, Constant(another))

    def __rmul__(self, another):
        if isinstance(another, Expression):
            return Expression(Mul,another,self)
        else:
            return Expression(Mul, Constant(another),self)
    
    def __truediv__(self, another):
        if isinstance(another, Expression):
            return Expression(Div,self,another)
        else:
            return Expression(Div, self, Constant(another))

    def __rtruediv__(self, another):
        if isinstance(another, Expression):
            return Expression(Div,another,self)
        else:
            return Expression(Div, Constant(another),self)
    
    def __pow__(self,another):
        if isinstance(another, Expression):
            return Expression(Pow,self,another)
        else:
            return Expression(Pow, self, Constant(another))
    
    def __rpow__(self,another):
        if isinstance(another, Expression):
            return Expression(Pow,another,self)
        else:
            return Expression(Pow, Constant(another),self)

In [390]:
class Variable(Expression):
    def __init__(self):
        return
    
    def evaluation_at(self, val_dict):
        return val_dict[self]
    
    def derivative_at(self, var,val_dict, order=1):
        if order ==1:
            return 1.0 if var is self else 0.0
        elif order ==2:
            return 0.0

In [391]:
class Constant(Expression):
    def __init__(self, val):
        self.val = val
        
    def evaluation_at(self, val_dict):
        return self.val
    
    def derivative_at(self, var, val_dict,order=1):
        return 0.0

In [392]:
class Neg:
    @staticmethod
    def evaluation_at(sub_expr1, val_dict):
        return -sub_expr1.evaluation_at(val_dict)
    
    @staticmethod
    def derivative_at(sub_expr1, var, val_dict):
        return -sub_expr1.derivative_at(var, val_dict)
             

In [393]:
class Add:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) + \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict,order=1):
        if order ==1: 
            return sub_expr1.derivative_at(var,val_dict) + \
               sub_expr2.derivative_at(var,val_dict)
        elif order==2:
            return sub_expr1.derivative_at(var,val_dict,2)+sub_expr2.derivative_at(var,val_dict,2)

In [394]:
class Sub:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) - \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict,order=1):
        if order ==1: 
            return sub_expr1.derivative_at(var,val_dict) - \
               sub_expr2.derivative_at(var,val_dict)
        elif order==2:
            return sub_expr1.derivative_at(var,val_dict,2)-sub_expr2.derivative_at(var,val_dict,2)

In [395]:
class Mul:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) *\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict,order=1):
        if order ==1:
            return sub_expr1.derivative_at(var, val_dict) * \
                   sub_expr2.evaluation_at(val_dict)+ \
                   sub_expr1.evaluation_at(val_dict) *\
                   sub_expr2.derivative_at(var, val_dict)
        elif order ==2:
            return sub_expr1.derivative_at(var, val_dict,2)*sub_expr2.evaluation_at(val_dict)+\
                   sub_expr1.derivative_at(var, val_dict,1)*sub_expr2.derivative_at(var, val_dict,1)+\
                   sub_expr1.derivative_at(var, val_dict,1)*sub_expr2.derivative_at(var, val_dict,1)+\
                   sub_expr1.evaluation_at(val_dict)*sub_expr2.derivative_at(var, val_dict,2)

In [402]:
class Div:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) /\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict,order=1):
        if order==1:
            return  sub_expr1.derivative_at(var, val_dict) / \
                    sub_expr2.evaluation_at(val_dict)- \
                    sub_expr1.evaluation_at(val_dict) *\
                    sub_expr2.derivative_at(var, val_dict)/\
                    sub_expr2.evaluation_at(val_dict)**2
        elif order==2:
            return ((sub_expr1.derivative_at(var, val_dict,2)*\
                    sub_expr2.evaluation_at(val_dict)-\
                    sub_expr1.evaluation_at(val_dict)*\
                    sub_expr2.derivative_at(var, val_dict,2))*sub_expr2.evaluation_at(val_dict)**2 -\
                    2*(sub_expr1.derivative_at(var, val_dict,1)*\
                    sub_expr2.evaluation_at(val_dict) -\
                    sub_expr1.evaluation_at(val_dict)*\
                    sub_expr2.derivative_at(var, val_dict,1))*\
                    sub_expr2.evaluation_at(val_dict)*\
                    sub_expr2.derivative_at(var, val_dict,1))/\
                    sub_expr2.evaluation_at(val_dict)**4
                
#              return sub_expr1.derivative_at(var, val_dict,2)/\
#                     sub_expr2.evaluation_at(val_dict)-\
#                     (sub_expr1.derivative_at(var, val_dict) /\
#                     sub_expr2.evaluation_at(val_dict)**2*\
#                     sub_expr2.derivative_at(var, val_dict))-\
#                     ((sub_expr1.derivative_at(var, val_dict)/\
#                     sub_expr2.evaluation_at(val_dict)**2*\
#                     sub_expr2.derivative_at(var, val_dict))-\
#                     (sub_expr1.evaluation_at(val_dict)/\
#                     2*sub_expr2.evaluation_at(val_dict)**3 *\
#                     sub_expr2.derivative_at(var, val_dict)**2)+\
#                     (sub_expr1.evaluation_at(val_dict)/\
#                     sub_expr2.evaluation_at(val_dict)**2*\
#                     sub_expr2.derivative_at(var, val_dict,2)))
                

In [403]:
# class Pow:
    
#     @staticmethod
#     def evaluation_at(sub_expr1, sub_expr2, val_dict):
#         return sub_expr1.evaluation_at(val_dict) **\
#                sub_expr2.evaluation_at(val_dict)
#     @staticmethod
#     #f(x)^g(x) * g‘(x)  * ln( f(x) )+ f(x)^( g(x)-1 ) * g(x) * f’(x) 
#     def derivative_at(sub_expr1, sub_expr2, var, val_dict):
#         return  sub_expr1.evaluation_at(val_dict)** \
#                 sub_expr2.evaluation_at(val_dict)* \
#                 sub_expr2.derivative_at(var, val_dict)*\
#                 np.log(sub_expr1.evaluation_at(val_dict))+ \
#                 sub_expr1.evaluation_at(val_dict) **\
#                 (sub_expr2.evaluation_at(val_dict)-1)*\
#                 sub_expr2.evaluation_at(val_dict)*\
#                 sub_expr1.derivative_at(var, val_dict)


# a simplified version: assuming sub_expr2 is a constant
class Pow:

    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return np.power(sub_expr1.evaluation_at(val_dict), 
                        sub_expr2.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict,order=1):
        p = sub_expr2.evaluation_at(val_dict)
        if order ==1:
            return p*np.power(sub_expr1.evaluation_at(val_dict), p-1.0) \
                   * sub_expr1.derivative_at(var, val_dict)
        elif order==2:
            return p*(p-1)*np.power(sub_expr1.evaluation_at(val_dict),p-2.0)* sub_expr1.derivative_at(var, val_dict)**2\
                    + p*np.power(sub_expr1.evaluation_at(val_dict), p-1.0)*sub_expr1.derivative_at(var, val_dict,2)

In [404]:
class Exp:
    @staticmethod
    def evaluation_at(sub_expr1, val_dict):
        return np.exp(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * np.exp(sub_expr1.evaluation_at(val_dict))

In [405]:
def exp(expr):
    return Expression(Exp, expr)

Let $f(a, b, c) = e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [414]:
x= Variable()
y = Variable()
f =x**3*y**3
g=(x/y)**3

$\dfrac{\partial f}{\partial b} = -e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $-e^{2} \approx -7.389$.

In [415]:
g.derivative_at(x, val_dict={x:2,y:3},order=2)

0.44444444444444442

In [416]:
g.derivative_at(y, val_dict={x:1,y:1},order=2)

12.0

In [409]:
6*2/27

0.4444444444444444

In [410]:
(12*1**3)/(1**5)

12.0

$\dfrac{\partial f}{\partial a} = e^{a-b+c}$, at $(a, b) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [411]:
f.derivative_at(a, {a: 1.0, b: 2.0, c: 3.0})

KeyError: <__main__.Variable object at 0x10afd1dd8>

Let $g(x, y) = x + e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $1+e \approx 3.718$.

In [20]:
x = Variable()
y = Variable()
g = x + exp(y-1)

In [62]:
g.evaluation_at({x: 1.0, y: 2.0})

3.7182818284590451

$\dfrac{\partial g}{\partial x} = 1$

In [63]:
g.derivative_at(x, {x: 1.0, y: 2.0})

1.0

$\dfrac{\partial g}{\partial y} = e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $e^{1} \approx 2.718$.

In [64]:
g.derivative_at(y, {x: 1.0, y: 2.0})

2.7182818284590451

#### Test for Multiple

In [65]:
x = Variable()
y = Variable()
g = x*y
f = x/y

In [66]:
g.evaluation_at({x: 2.0, y: 2.0})

4.0

In [67]:
g.derivative_at(x, {x: 1.0, y: 2.0})

2.0

In [68]:
f.evaluation_at({x: 3.0, y: 2.0})

1.5

In [69]:
f.derivative_at(y, {x: 4.0, y: 2.0})

1.0

In [70]:
a = 3.4

In [71]:
2/3

0.6666666666666666

### Test for Power

In [21]:
x = Variable()
y = Variable()
g = x**y
f = x**2+x**0.5

In [40]:
g.evaluation_at({x: 3.0, y: 2.0})

9.0

In [74]:
f.derivative_at(x, {x: 3.0})

6.2886751345948131

In [34]:
0.5*3**(-0.5)

0.28867513459481287

In [43]:
g.derivative_at(x, {x: 3.0, y: 2.0})

6.0

In [44]:
np.log(0.1)

-2.3025850929940455

### Test for unary

In [4]:
x=Variable()
f= -x

NameError: name 'Variable' is not defined

In [45]:
f.evaluation_at({x:2,y:3})

-2

In [46]:
f.derivative_at(x,2,{x:2,y:3})

-1.0

In [28]:
# Import their package


ModuleNotFoundError: No module named 'autodiff'

NameError: name 'sin' is not defined